In [2]:
import pandas as pd

df = pd.read_csv("materials_milestone1_final.csv")
df.head()


,id,material,weight,cost,co2_emission,durability,recyclable,co2_impact_index,cost_efficiency_index,sustainability_score
0,1,Recycled Cardboard,44.55,3.95,0.841,7,0,0.561415,0.095386,0.408767
1,2,Corn Starch Polymer,42.85,4.91,0.702,1,0,0.468625,0.000125,0.245883
2,3,Recycled Cardboard,38.48,4.22,0.931,8,0,0.621495,0.102892,0.418069
3,4,Recycled Paperboard,10.84,1.34,0.837,8,1,0.558745,0.350350,0.743169
4,5,Recycled Paperboard,20.44,3.93,1.054,1,1,0.703605,0.003209,0.451891


In [3]:
y_cost = df['cost']
y_co2  = df['co2_emission']


In [4]:
features = [
    'weight',
    'durability',
    'recyclable',
    'co2_impact_index',
    'cost_efficiency_index',
    'sustainability_score'
]

X = df[features]


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_cost_train, y_cost_test = train_test_split(
    X, y_cost, test_size=0.2, random_state=42
)

_, _, y_co2_train, y_co2_test = train_test_split(
    X, y_co2, test_size=0.2, random_state=42
)


In [6]:
from sklearn.ensemble import RandomForestRegressor

rf_cost = RandomForestRegressor(
    n_estimators=100,
    random_state=42
)

rf_cost.fit(X_train, y_cost_train)


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

In [7]:
rf_co2 = RandomForestRegressor(
    n_estimators=100,
    random_state=42
)

rf_co2.fit(X_train, y_co2_train)


,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

In [8]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Cost predictions
cost_pred = rf_cost.predict(X_test)

# CO2 predictions
co2_pred = rf_co2.predict(X_test)

def evaluate(y_true, y_pred, name):
    print(f"\n{name}")
    print("MAE :", mean_absolute_error(y_true, y_pred))
    print("RMSE:", np.sqrt(mean_squared_error(y_true, y_pred)))
    print("R2  :", r2_score(y_true, y_pred))

evaluate(y_cost_test, cost_pred, "Cost Prediction Model")
evaluate(y_co2_test, co2_pred, "CO2 Prediction Model")



Cost Prediction Model
MAE : 0.11825599999999964
RMSE: 0.166629735041498
R2  : 0.984133558360042

CO2 Prediction Model
MAE : 0.0032603000000000883
RMSE: 0.004262639909727405
R2  : 0.9998849048470798


In [9]:
-df['predicted_cost'] = rf_cost.predict(X)
df['predicted_co2']  = rf_co2.predict(X)

df['final_rank_score'] = (
    0.5 * (1 - df['predicted_co2']/df['predicted_co2'].max()) +
    0.5 * (1 - df['predicted_cost']/df['predicted_cost'].max())
)

df.sort_values(by='final_rank_score', ascending=False).head(10)


,id,material,weight,cost,co2_emission,durability,recyclable,co2_impact_index,cost_efficiency_index,sustainability_score,predicted_cost,predicted_co2,final_rank_score
132,133,Biodegradable Plastic,36.04,0.86,0.099,6,1,0.066088,0.411486,0.873565,0.8521,0.10311,0.878681
351,352,Corn Starch Polymer,5.33,0.84,0.095,1,1,0.063418,0.060058,0.707966,0.9194,0.09884,0.873248
258,259,Aluminum,14.83,0.86,0.115,3,1,0.076769,0.199620,0.769292,0.9335,0.11190,0.867446
403,404,Bagasse Fiber,51.56,0.72,0.218,6,0,0.145527,0.493878,0.541789,0.7471,0.21762,0.851115
320,321,Bagasse Fiber,34.50,1.02,0.157,9,1,0.104806,0.523649,0.958077,0.9647,0.15720,0.849126
229,230,Glass,59.01,1.42,0.066,7,0,0.044059,0.287151,0.615710,1.3895,0.06697,0.835980
62,63,PLA Bioplastic,24.60,0.87,0.230,4,1,0.153538,0.266995,0.771918,0.8769,0.22611,0.835046
118,119,Bagasse Fiber,52.91,0.55,0.313,7,0,0.208945,0.760742,0.549755,0.6114,0.31089,0.833776
435,436,Aluminum,37.17,1.03,0.216,5,1,0.144192,0.282584,0.808990,0.9777,0.21659,0.827953
206,207,Aluminum,16.08,1.09,0.184,3,0,0.122830,0.154915,0.450868,1.1099,0.18531,0.824931
